# Задачи

## Банкови заеми

Банка е в процеса на определяне на кредитната си политика за пет
различни сектора с характеристики дадени в следната таблица:

| Вид кредит      | Лихва | Дял на несъбираеми кредити |
|-----------------|------:|---------------------------:|
| Потребителски   |  0.15 |                       0.10 |
| Автомобили      |  0.13 |                       0.07 |
| Недвижими имоти |  0.12 |                       0.03 |
| Земеделие       | 0.125 |                       0.05 |
| Търговия        |  0.10 |                       0.02 |

Банката разполага с общо 12 милиона лева и при определяне на политиката
трябва да се съобрази със следните регулации:

1. Поне 40% от заемите трябва да са насочени към търговския сектор.
2. Заеми за недвижими имоти трябва да са поне половината от общата сума на потребителски кредити, кредити свързани с автомобили и недвижими имоти.
3. По силата на регулаторни изисквания делът на несъбираемите кредити не трябва да надвишава 4% от всички кредити. Несъбираемите кредити не носят лихви.

Какъв е оптималният (най-висока печалба) план за банката?


## Производство и склад (1)

Малък шивашки цех се опитва да определи производствения си план за следващия сезон. Цехът произвежда
четири продукта: якета, шапки, ръкавици и панталони, които се произвеждат в четири отделения: рязане, уплътняване, шиене и пакетиране.


| Отделения/продукти | Якета | Шапки | Ръкавици | Панталони | Капацитет (часове) |
|--------------------|------:|------:|---------:|----------:|-------------------:|
| Рязане             |   0.3 |   0.3 |     0.25 |      0.15 |               1000 |
| Уплътняване        |  0.25 |  0.35 |      0.3 |      0.10 |               1000 |
| Шиене              |  0.45 |   0.5 |      0.4 |      0.22 |               1000 |
| Пакетиране         |  0.12 |  0.15 |      0.1 |      0.05 |               1000 |


Цехът вече има получени поръчки за четирите продукта. Договорите за производство включват неустойка за всяка недоставена единица:

| Отделения/продукти   | Якета | Шапки | Ръкавици | Панталони |
|----------------------|------:|------:|---------:|----------:|
| Поръчки (брой)       |   800 |   750 |      600 |       600 |
| Печалба (лв./брой)   |    30 |    40 |       20 |        10 |
| Неустойка (лв./брой) |    15 |    20 |       10 |         8 |


Намерете оптималния производствен план за този цех.


## Производство и склад (1)

Компания за производство на дограма има договор за доставка на дървени дограми за следващите 6 месеца. За същия период компанията очаква разходите за производство (труд и материали) да варират. Компанията може да се възползва от по-ниски производствени разходи в даден период и да произведе повече дограми от необходимото, като разходите за складиране на продуктите са 8 лв. на месец на брой. В момента на планиране фирмата няма готови дограми на склад.

|                                   | Януари | Февруари | Март | Април | Май | Юни |
|-----------------------------------|-------:|---------:|-----:|------:|-----|-----|
| Поръчки (брой)                    |    100 |      250 |  190 |   140 | 220 | 110 |
| Производствени разходи (лв./брой) |     50 |       45 |   55 |    48 | 52  | 50  |


Формулирайте оптимизационна задача, която да намери оптималния производствен план.


## Производство и персонал (1)

Компания за производство на дограма има договор за доставка на дървени дограми за следващата година.
Компанията разполага с 10 опитни сътрудника, назначени на трудови договори. Всеки от тях произвежда по 12 дограми на тримесечие. Компанията може да отговори на флуктуациите в търсенето на своите продукти, като назначава и уволнява сътрудници на срочни договори. Временно заетите произвеждат по 3 дограми на месец. Разходите за назначаване/уволняване на временни сътрудници са съответно 200 лв. и 400 лв. за тримесечие. Компанията може да складира непродадени единици на цена 50 лв. на месец на единица.

|                | Януари-Март | Април-Юни | Юли-Септември | Октомври-Декември |
|----------------|------------:|----------:|--------------:|------------------:|
| Поръчки (брой) |         520 |       720 |           520 |               620 |


Формулирайте оптимизационна задача, която да намери оптималния план за назначаване/уволняване на временно заетите работници.



## Производство и персонал (2)

Фирма, специализирана в производство на зимни палта се опитва да посрещне
търсенето на нейните стоки с възможно най-малки разходи. Всеки работник, нает в началото на
сезона работи през целия сезон, произвежда 80 палта и получава 6000 лв. заплата.
В началото на годината фирмата разполага с 30 работника.

Търсенето на палта има силни сезонни колебания и фирмата има три опции,
за да отговори на тези  вариации:

- работниците могат да работят и по-продължително, но допълнителният труд се
заплаща 50% по-скъпо и работниците могат да работят не повече от 20% повече
от нормалното работно време.
- Произведени в предишния период палта могат да се складират на цена от 12 лв.
на сезон за всяко палто.
- Фирмата може да назначава и освобождава служители. За назначаване на един
нов служител цената е 500 лв. За освобождаване на служител разходите възлизат
на 420 лв.
- Фирмата няма начални запаси от готови палта.


Маркетинговият отдел на предприятието оценява, че през следващите четири сезона
търсенето ще възлиза на съответно 5000, 2000, 500 и 1500 палта.

Изгответе математически модел с който да намерите оптималния план
(в смисъл на най-ниски разходи) за тази фирма.
